### **Modeling Development and Tracking Experiments with mlflow**

In [1]:
# Import Libraries

import pandas as pd
import numpy as np

import math
from scipy.stats import randint, uniform
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import metrics

import seaborn as sns

import pickle
from pickle import dump
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
import mlflow
import mlflow.sklearn
import mlflow.pyfunc

In [4]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Laptop Prices Prediction")

<Experiment: artifact_location='file:///c:/Users/ADMIN/Desktop/Predict-Laptop-Prices-Streamlit/notebooks/mlruns/2', creation_time=1684007265560, experiment_id='2', last_update_time=1684007265560, lifecycle_stage='active', name='Laptop Prices Prediction', tags={}>

In [5]:
# Load data

df = pd.read_csv('..\data\cleaned_data2.csv')
df.head()

,MRP,RAM_Size,RAM_Type,Display,Processor,Storage,OS,Brand
0,36990,8,DDR4,14.0,Intel Core i3,256 GB SSD,Windows,Lenovo
1,39990,8,DDR4,15.6,Intel Core i3,512 GB SSD,Windows,Lenovo
2,32990,8,DDR4,15.6,Intel Core i3,512 GB SSD,Windows,ASUS
3,49990,8,DDR4,15.6,AMD Ryzen 5 Hexa Core,512 GB SSD,Windows,HP
4,49990,8,DDR4,15.6,Intel Core i5,512 GB SSD,Windows,ASUS


In [6]:
df.shape

(423, 8)

In [7]:
# Generate dictionaries of the categorical columns

def generate_dicts(df):
    categorical_cols = df.select_dtypes(include=['object']).columns
    dicts_cols = {}
    for col in categorical_cols:
        unique_values = df[col].unique()
        dict_col = {}
        for i, value in enumerate(sorted(unique_values)):
            dict_col[value] = i
        dicts_cols[col] = dict_col
    return dicts_cols

dicts_cols = generate_dicts(df)
print(dicts_cols)

# Replace the string values with the corresponding integer values

df = df.replace(dicts_cols)

{'RAM_Type': {'DDR4': 0, 'DDR5': 1, 'LPDDR3': 2, 'LPDDR4': 3, 'LPDDR4X': 4, 'LPDDR5': 5, 'Unified Memory': 6}, 'Processor': {'AMD Athlon Dual Core': 0, 'AMD Ryzen 3': 1, 'AMD Ryzen 3 Dual Core': 2, 'AMD Ryzen 3 Hexa Core': 3, 'AMD Ryzen 3 Quad Core': 4, 'AMD Ryzen 5': 5, 'AMD Ryzen 5 Dual Core': 6, 'AMD Ryzen 5 Hexa Core': 7, 'AMD Ryzen 5 Quad Core': 8, 'AMD Ryzen 7 Octa Core': 9, 'AMD Ryzen 7 Quad Core': 10, 'AMD Ryzen 9 Octa Core': 11, 'Intel Celeron Dual Core': 12, 'Intel Celeron Quad Core': 13, 'Intel Core i3': 14, 'Intel Core i5': 15, 'Intel Core i7': 16, 'Intel Core i9': 17, 'Intel Evo Core i5': 18, 'Intel Pentium Quad Core': 19, 'Intel Pentium Silver': 20, 'M1': 21, 'M1 Max': 22, 'M1 Pro': 23, 'M2': 24, 'Qualcomm Snapdragon 7c Gen 2': 25}, 'Storage': {'1 TB HDD': 0, '1 TB HDD, 128 GB SSD': 1, '1 TB HDD, 256 GB SSD': 2, '1 TB HDD, 512 GB SSD': 3, '128 GB SSD': 4, '256 GB SSD': 5, '512 GB SSD': 6}, 'OS': {'Chrome': 0, 'DOS': 1, 'Mac OS': 2, 'Windows': 3}, 'Brand': {'ALIENWARE': 0,

In [8]:
# Separate the target variable from the features

X = df.drop(columns=['MRP'], axis=1)
y = np.log(df['MRP'])

In [9]:
# Split data into train and test 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Rescale RAM Size and Display columns only

scaler = MinMaxScaler()

X_train_scaled[['Display', 'RAM_Size']] = scaler.fit_transform(X_train_scaled[['Display', 'RAM_Size']])
X_test_scaled[['Display', 'RAM_Size']] = scaler.transform(X_test_scaled[['Display', 'RAM_Size']])

# Create new dataframes with the scaled data and original column names

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

### **Experiment Tracking**

In [12]:
mlflow.autolog()

2023/05/15 16:22:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/05/15 16:22:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


**XGBoost Regressor**

In [14]:
with mlflow.start_run():

    mlflow.set_tag("algorithm", "XGBoost")

    # log the data for each run using log_param, log_metric, log_model
    mlflow.log_param("data-path", "..\data\cleaned_data2.csv")

    # Set the hyperparameters
    hyperparams = {
        'learning_rate': 0.2,
        'n_estimators': 200,
        'max_depth': 50
    }
    mlflow.log_params(hyperparams)

    xgbr = XGBRegressor(**hyperparams)
    xgbr.fit(X_train, y_train)

    y_pred = xgbr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(xgbr, artifact_path="models")
    
    mlflow.end_run()

**GradientBoosting Regressor**

In [18]:
with mlflow.start_run():

    mlflow.set_tag("algorithm", "Gradient Boost")

    # log the data for each run using log_param, log_metric, log_model
    mlflow.log_param("data-path", "..\data\cleaned_data2.csv")
    
    learning_rate = 0.3
    mlflow.log_param("learning_rate", learning_rate)
    
    gbr = GradientBoostingRegressor(learning_rate = learning_rate)
    gbr.fit(X_train, y_train)

    y_pred = gbr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(gbr, artifact_path="models")
    
    mlflow.end_run()

**Random Forest Regressor**

In [16]:
with mlflow.start_run():

    mlflow.set_tag("algorithm", "Random Forest")

    # log the data for each run using log_param, log_metric, log_model
    mlflow.log_param("data-path", "..\data\cleaned_data2.csv")

    n_estimators = 200
    mlflow.log_param("n_estimators", n_estimators)

    rf = RandomForestRegressor(n_estimators)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    
    mlflow.sklearn.log_model(rf, artifact_path="models")
    mlflow.end_run()

### **Hyperparameter Tuning with hyperopt**

XGBoost Regressor

In [23]:
mlflow.xgboost.autolog()

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("algorithm", "XGBoost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
       
    return {'loss': rmse, 'status': STATUS_OK}

In [25]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 200, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1)
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.84760                           
[1]	validation-rmse:9.10898                           
[2]	validation-rmse:8.42643                           
[3]	validation-rmse:7.79575                           
[4]	validation-rmse:7.21302                           
[5]	validation-rmse:6.67439                           
[6]	validation-rmse:6.17654                           
[7]	validation-rmse:5.71640                           
[8]	validation-rmse:5.29116                           
[9]	validation-rmse:4.89819                           
[10]	validation-rmse:4.53508                          
[11]	validation-rmse:4.19961                          
[12]	validation-rmse:3.88970                          
[13]	validation-rmse:3.60696                          
[14]	validation-rmse:3.34006                          
[15]	validation-rmse:3.09904                          
[16]	validation-rmse:2.87365                          
[17]	validation-rmse:2.66602                          
[18]	valid

GradientBoosting Regressor

In [13]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("algorithm", "GradientBoost")
        mlflow.log_params(params)

        lr = params['learning_rate']
        n_estimators = params['n_estimators']
        max_depth = params['max_depth']

        gboost = GradientBoostingRegressor(learning_rate=lr, n_estimators=n_estimators, max_depth=max_depth)
        gboost.fit(X_train, y_train)
        y_pred = gboost.predict(X_test)

        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
       
    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
   'learning_rate': hp.loguniform('learning_rate', -5, 2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300]),
    'max_depth': hp.choice('max_depth', [3, 5, 8, 10, 15, 20])
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

100%|██████████| 50/50 [09:28<00:00, 11.37s/trial, best loss: 0.21527263726643833]  


#### **Train the model with the attained hyperparameters**

In [12]:
mlflow.autolog(disable=True)

In [14]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_test, label=y_test)

    best_params = {
    'learning_rate' : 0.22720596613069743,
    'max_depth' : 120,
    'reg_alpha' : 0.10673584255582483,
    'custom_metric' : None,
    'maximize' : None,
    'reg_alpha' : 0.045498626340112994,
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    with open('../models/dicts_cols.pkl', 'wb') as f_out:
        pickle.dump(dicts_cols, f_out)
    mlflow.log_artifact("../models/dicts_cols.pkl", artifact_path="preprocessor")

    with open('../models/scaler.pkl', 'wb') as f_out:
        pickle.dump(scaler, f_out)
    mlflow.log_artifact("../models/scaler.pkl", artifact_path="preprocessor")
        
    mlflow.xgboost.log_model(booster, artifact_path="models")

[23:44:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "early_stopping_rounds", "num_boost_round", "verbose_eval" } are not used.

[0]	validation-rmse:8.25447
[1]	validation-rmse:6.40621
[2]	validation-rmse:4.97705
[3]	validation-rmse:3.87278
[4]	validation-rmse:3.02074
[5]	validation-rmse:2.36715
[6]	validation-rmse:1.86438
[7]	validation-rmse:1.47463
[8]	validation-rmse:1.17025
[9]	validation-rmse:0.93770
[10]	validation-rmse:0.76092
[11]	validation-rmse:0.61843
[12]	validation-rmse:0.51307
[13]	validation-rmse:0.43028
[14]	validation-rmse:0.36785
[15]	validation-rmse:0.32264
[16]	validation-rmse:0.28817
[17]	validation-rmse:0.26332
[18]	validation-rmse:0.24546
[19]	validation-rmse:0.23255
[20]	validation-rmse:0.22373
[21]	validation-rmse:0.21882
[22]	validation-rmse:0.21556
[23]	validation-rmse:0.21320
[24]	validation-rmse:0.21139
[25]	validation-rmse:0.21001
[26]	valid

#### **Save the model**

In [15]:
logged_model = 'runs:/6107df096d2c4d68b62dd73dff64030a/models'

# Save the model as an xgboost flavour
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

In [16]:
y_pred = xgboost_model.predict(valid)
y_pred = np.exp(y_pred)
y_pred

array([103169.79 ,  90803.   ,  64852.28 ,  75039.15 ,  73846.4  ,
        37123.062,  52857.492,  60951.32 , 167502.14 ,  65899.125,
        53092.06 ,  37016.613,  69316.984,  43008.547,  52857.492,
       134755.89 ,  98240.65 ,  76374.08 , 240193.05 , 247271.08 ,
        50523.145,  62131.61 ,  79788.69 ,  35704.066,  60951.32 ,
       141036.42 ,  57600.664,  35016.258,  74258.195, 103169.79 ,
        69343.7  ,  20357.574,  57503.52 ,  42386.406,  66695.9  ,
        50791.46 ,  58901.883,  59067.035,  58511.094,  67839.82 ,
        79366.66 , 128780.09 , 109382.15 , 182601.2  ,  73264.46 ,
        43539.594,  45159.184,  65899.125, 244960.83 , 182601.2  ,
        54356.754,  80173.96 ,  39356.438, 132044.28 ,  71385.414,
       142849.62 ,  51116.65 ,  36405.906, 120425.17 ,  37016.613,
       126624.4  ,  49779.098,  69316.984,  57429.367,  71079.12 ,
        60864.543,  48314.273,  60708.652,  64547.84 ,  98295.19 ,
        17586.438,  37717.52 ,  50169.14 ,  22082.688,  21829.